In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import mutagen
import os
import random
import wave
import numpy as np
from scipy.io.wavfile import read as read_wav
from scipy.io.wavfile import write as write_wav
import librosa
import librosa.display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

import utils
import classifier
import multi_classifier as mclass
import irmasTrainUtils as trainUtils
import irmasTestUtils as testUtils

In [4]:
trainUtils.parse_irmas_trainset("IRMAS-TrainingData", "Preprocessed_Trainset")

In [5]:
df_train = trainUtils.load_train_dataset("Preprocessed_Trainset/Train")
df_valid = trainUtils.load_train_dataset("Preprocessed_Trainset/Validation")

In [6]:
df_train.to_pickle("Preprocessed_Trainset/pickled_train.pkl")
df_valid.to_pickle("Preprocessed_Trainset/pickled_validation.pkl")

In [5]:
df_train = utils.load_pickled_dataset("../train.pkl")
df_valid = utils.load_pickled_dataset("../validation.pkl")
df_train = df_train.sample(frac=1)
df_valid = df_valid.sample(frac=1)

In [4]:
model = classifier.make_model()

In [9]:
classifier.train_model(model, df_train, batch_size=32, epochs=30)

In [5]:
model.load_weights("Saved_Weights/weights")

In [6]:
print(classifier.get_accuracy(model, df_train))

[3]
0.7036668325866932


In [7]:
print(classifier.get_accuracy(model, df_valid))

[3]
0.6047197640117994


In [ ]:
print(model.history)

In [6]:
multi_model = mclass.make_model()
#mclass.load_weights(multi_model, model)
multi_model.load_weights("Saved_Weights/multi_frozen")

In [7]:
mclass.train_model(multi_model, df_train, batch_size=32, epochs=30)

Epoch 1/30
377/377 [==============================] - 56s 142ms/step - loss: 2.2876
Epoch 2/30
377/377 [==============================] - 54s 144ms/step - loss: 1.6352
Epoch 3/30
377/377 [==============================] - 56s 150ms/step - loss: 1.4381
Epoch 4/30
377/377 [==============================] - 59s 155ms/step - loss: 1.3540
Epoch 5/30
377/377 [==============================] - 65s 173ms/step - loss: 1.2586
Epoch 6/30
377/377 [==============================] - 64s 169ms/step - loss: 1.2007
Epoch 7/30
377/377 [==============================] - 64s 170ms/step - loss: 1.1377
Epoch 8/30
377/377 [==============================] - 63s 168ms/step - loss: 1.1310
Epoch 9/30
377/377 [==============================] - 63s 167ms/step - loss: 1.0848
Epoch 10/30
377/377 [==============================] - 60s 159ms/step - loss: 1.0903
Epoch 11/30
377/377 [==============================] - 56s 149ms/step - loss: 1.0363
Epoch 12/30
377/377 [==============================] - 62s 164ms/step - lo

In [7]:
print(mclass.get_accuracy(multi_model, df_train))
print(mclass.get_accuracy(multi_model, df_valid))

0.3996999612853271
0.31492133726647004


In [8]:
# Train set accuracy (Multi Class)

print(mclass.get_accuracy(multi_model, df_train))

0.39969996128532714


In [9]:
# Validation set accuracy (Multi class)

print(mclass.get_accuracy(multi_model, df_valid))

0.31492133726647004


In [12]:

x, y = np.stack(df_train.data), np.stack(df_train.label)
x = x.reshape((-1,128,259,2))
print((multi_model.predict(x)[0] > 0.5) * 1)

[0.86317337 0.36226666 0.0948779  0.14307916 0.30608308 0.06918737
 0.21820486 0.82087505 0.53484875 0.88276625 0.01191196]


In [10]:
multi_model.save_weights("Saved_Weights/multi_frozen")


In [8]:
df_test = utils.load_pickled_dataset("../preprocessed_test.pkl")

In [9]:
df_test.head(2)

,file,dir,root,rate,data,nframes,nchannels,label
0,[00000000001] 06 - Lali Puna - Faking the Book...,[],/Users/Dominick/Documents/STANFORD/CS 230/Prep...,44100,"[[[-37.234799641383894, -37.78036096357812], [...",132299,2,"[[0], [0], [0], [0], [0], [0], [0], [0], [0], ..."
1,[00001000001] 01. Helmet - Pure-4.wav,[],/Users/Dominick/Documents/STANFORD/CS 230/Prep...,44100,"[[[-58.7643634758266, -31.256983821161835], [-...",132299,2,"[[0], [0], [0], [0], [1], [0], [0], [0], [0], ..."


In [10]:
df_valid.head(2)

,file,dir,root,rate,data,nframes,nchannels,label,instruments
680,[tru][jaz_blu]1884__3[1].wav,[],/Users/Dominick/Documents/STANFORD/CS 230/Prep...,44100,"[[[-62.61958715422706, -62.61958715422706], [-...",132299,2,"[[0], [0], [0], [0], [0], [0], [0], [0], [1], ...",tru
529,[pia][jaz_blu]1515__1[1].wav,[],/Users/Dominick/Documents/STANFORD/CS 230/Prep...,44100,"[[[-46.34581292333955, -55.61510248144694], [-...",132299,2,"[[0], [0], [0], [0], [0], [0], [1], [0], [0], ...",pia


In [11]:
# Test set accuracy (Multi Class)

print(mclass.get_accuracy(multi_model, df_test))

0.3049484430747479
